In [21]:
import pandas as pd

In [2]:

df = pd.read_csv('./Restaurant_revenue (1).csv')
df.head()

,Number_of_Customers,Menu_Price,Marketing_Spend,Cuisine_Type,Average_Customer_Spending,Promotions,Reviews,Monthly_Revenue
0,61,43.117635,12.663793,Japanese,36.236133,0,45,350.912040
1,24,40.020077,4.577892,Italian,17.952562,0,36,221.319091
2,81,41.981485,4.652911,Japanese,22.600420,1,91,326.529763
3,70,43.005307,4.416053,Italian,18.984098,1,59,348.190573
4,30,17.456199,3.475052,Italian,12.766143,1,30,185.009121


In [3]:
# Code generated by Data Wrangler for pandas DataFrame

def clean_data(df):
    # Select column: 'Cuisine_Type'
    df = df.loc[:, ['Cuisine_Type']]
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    return df

df_clean = clean_data(df.copy())

In [4]:
df_clean.head()

,Cuisine_Type
0,Japanese
1,Italian
5,American
10,Mexican


In [6]:
df_clean.reset_index(drop=True, inplace=True)

In [7]:
df_clean['CuisineTypeID'] = range(0, 0+len(df_clean)) 

df_clean

,Cuisine_Type,CuisineTypeID
0,Japanese,0
1,Italian,1
2,American,2
3,Mexican,3


In [8]:
fact = df.merge(df_clean, on='Cuisine_Type', how='left')

In [9]:
fact.head()

,Number_of_Customers,Menu_Price,Marketing_Spend,Cuisine_Type,Average_Customer_Spending,Promotions,Reviews,Monthly_Revenue,CuisineTypeID
0,61,43.117635,12.663793,Japanese,36.236133,0,45,350.912040,0
1,24,40.020077,4.577892,Italian,17.952562,0,36,221.319091,1
2,81,41.981485,4.652911,Japanese,22.600420,1,91,326.529763,0
3,70,43.005307,4.416053,Italian,18.984098,1,59,348.190573,1
4,30,17.456199,3.475052,Italian,12.766143,1,30,185.009121,1


In [12]:
def clean_data(fact):
    fact = fact.drop(columns=['Cuisine_Type'])
    return fact



In [ ]:

fact = clean_data(fact.copy())

In [13]:

fact.head()

,Number_of_Customers,Menu_Price,Marketing_Spend,Average_Customer_Spending,Promotions,Reviews,Monthly_Revenue,CuisineTypeID
0,61,43.117635,12.663793,36.236133,0,45,350.912040,0
1,24,40.020077,4.577892,17.952562,0,36,221.319091,1
2,81,41.981485,4.652911,22.600420,1,91,326.529763,0
3,70,43.005307,4.416053,18.984098,1,59,348.190573,1
4,30,17.456199,3.475052,12.766143,1,30,185.009121,1


In [14]:
fact.tail()

,Number_of_Customers,Menu_Price,Marketing_Spend,Average_Customer_Spending,Promotions,Reviews,Monthly_Revenue,CuisineTypeID
995,73,41.307842,12.122931,19.033585,1,40,249.312034,0
996,31,20.615496,5.822885,17.040990,0,57,110.228767,3
997,69,17.110656,4.141898,44.649315,0,55,312.212552,0
998,73,37.664722,3.046556,27.767358,0,23,272.482204,0
999,81,34.722067,17.989104,15.482112,1,72,379.973072,1


In [15]:
def clean_data(fact):
    fact = fact.drop(columns=['Number_of_Customers','Menu_Price','Marketing_Spend','Average_Customer_Spending','Reviews','Monthly_Revenue','CuisineTypeID'])
    fact = fact.drop_duplicates()
    return fact

prom = clean_data(fact.copy())
prom.head()

,Promotions
0,0
2,1


In [16]:
prom['PromotionsName'] = prom['Promotions'].map({0: 'No', 1: 'Yes'})
prom.head()

,Promotions,PromotionsName
0,0,No
2,1,Yes


In [ ]:
spark_df_cuisine = spark.createDataFrame(df_clean)
display(spark_df_cuisine)

In [ ]:
spark_df_fact = spark.createDataFrame(fact)
spark_df_prom = spark.createDataFrame(prom)

In [ ]:
spark_df_cuisine.write.format("delta").mode("overwrite").saveAsTable("DimCuisineType")
spark_df_prom.write.format("delta").mode("overwrite").saveAsTable("DimPromotion")
spark_df_fact.write.format("delta").mode("overwrite").saveAsTable("Fact")


In [ ]:
spark_df = spark.createDataFrame(df)
display(spark_df)

In [ ]:
spark_df.write.format("delta").mode("overwrite").saveAsTable("sales_raw")

#### Switch to SQL Analytics 
Run queries to create views and queries

```sql
-- Create a View for Cuisine Type Dimension Save as vDimCuisineType
SELECT 
    CuisineTypeID = ROW_NUMBER() OVER (ORDER BY Cuisine_Type), Cuisine_Type
FROM
    (
        SELECT DISTINCT Cuisine_Type 
        FROM sales_raw
    ) AS DimCuisineType

-- Create a View for Promotion Dimension Save as vPromotionName
SELECT DISTINCT Promotion AS Promotion 
CASE
    WHEN Promotion = 0 THEN 'No Promotion'
    WHEN Promotion = 1 THEN 'Promotion'
    ELSE 'unknown'
END AS PromotionName
FROM sales_raw

-- Create a View for the FactTable Save as vFactRestaurant
select Number_of_Customers,	Menu_Price,	Marketing_Spend,	Average_Customer_Spending,	Promotions,	Reviews,	Monthly_Revenue,b.CuisineTypeID from sales_raw a
left join DimCuisineType b
on a.Cuisine_Type = b.Cuisine_Type
```

#### Create New Semantic Model in Lakehouse
- Calculation groups
- Cultures
- Create many-to-one relationship
- Create measures
- Perspectives
- Roles
- Tables

#### Create New PowerBI Report

- Fact by cuisine type, sum, value label (CHART)
- Customer spend, sum, value label(CHART)
- Total spend as a value
- Menu price sum
- Average menu price